# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new plymouth,6.6518,-10.5932,22.77,98,93,1.43,LR,1691548056
1,1,shubarshi,48.5872,57.1869,21.89,56,7,2.67,KZ,1691548056
2,2,bredasdorp,-34.5322,20.0403,8.80,90,0,2.70,ZA,1691548056
3,3,west island,31.8024,-97.0917,35.86,34,0,4.12,US,1691548056
4,4,wailua homesteads,22.0492,-159.3352,34.00,77,100,9.77,US,1691548057


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:


# Configure the map plot
city_humidity = {
    "Lat": city_data_df["Lat"],
    "Lng": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity": city_data_df["Humidity"]
}
city_humidity_df = pd.DataFrame(city_humidity)
humidity_plot = city_humidity_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_height= 600, frame_width= 800, size = "Humidity", color = "City")


# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 35) & (city_data_df["Max Temp"] > 17) & (city_data_df["Cloudiness"] < 30) & (city_data_df["Humidity"] < 35)]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,tejen,37.3834,60.5054,28.80,24,0,2.55,TM,1691548062
152,152,murzuq,25.9155,13.9184,27.17,30,0,4.64,LY,1691548078
157,157,kokuy,52.2037,117.5529,27.32,32,19,1.48,RU,1691548079
161,161,illizi,26.4833,8.4667,28.89,21,11,6.33,DZ,1691548301
175,175,sirjan,29.4520,55.6814,24.54,33,16,1.11,IR,1691548082


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
43,tejen,TM,37.3834,60.5054,24,
152,murzuq,LY,25.9155,13.9184,30,
157,kokuy,RU,52.2037,117.5529,32,
161,illizi,DZ,26.4833,8.4667,21,
175,sirjan,IR,29.4520,55.6814,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
hotels = ["accommodation.hotel"]
params = {
    "categories" : hotels,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity {lat},{lng}"
    
    # Set base URL
    
    base_url = "https://api.geoapify.com/v1/geocode/search?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # print(name_address)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # print(name_address)
    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
tejen - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
kokuy - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
sirjan - nearest hotel: No hotel found
arraias - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
bend - nearest hotel: No hotel found
nushki - nearest hotel: No hotel found
kaedi - nearest hotel: No hotel found
turpan - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
baharly - nearest hotel: No hotel found
butte - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
afif - nearest hotel: No hotel found
kaindy - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
hobbs - nearest hotel: No hotel found
mammoth lakes - nearest hotel: No hotel found
ravar - nearest hotel: No hotel found
pirapora - nearest hotel: No hotel found
medina - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
43,tejen,TM,37.3834,60.5054,24,No hotel found
152,murzuq,LY,25.9155,13.9184,30,No hotel found
157,kokuy,RU,52.2037,117.5529,32,No hotel found
161,illizi,DZ,26.4833,8.4667,21,No hotel found
175,sirjan,IR,29.4520,55.6814,33,No hotel found
179,arraias,BR,-12.9288,-46.9437,28,No hotel found
188,karratha,AU,-20.7377,116.8463,26,No hotel found
245,bend,US,44.0582,-121.3153,26,No hotel found
249,nushki,PK,29.5542,66.0215,17,No hotel found
288,kaedi,MR,16.1465,-13.5047,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:


# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", color = "City", hover_cols = ["Hotel Name", "Country"], frame_height = 800, frame_width = 1000)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)